<a href="https://colab.research.google.com/github/sayanbanerjee32/ms-phi2-qlora-oasst1/blob/main/fine_tuned_model_diff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers==4.44.2
!pip install -Uq peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 3.3 MB/s eta 0:00:00


In [2]:
import torch
from transformers import AutoModelForCausalLM
from peft import PeftModel
import numpy as np
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

In [3]:
def load_model(model_path, device='cpu'):
    return AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float32, low_cpu_mem_usage=True).to(device)

def compare_models(base_model, fine_tuned_model):
    differences = []
    for (name1, p1), (name2, p2) in zip(base_model.named_parameters(), fine_tuned_model.named_parameters()):
        if name1 == name2:
            diff = torch.norm(p1 - p2).item()
            differences.append((name1, diff))
    return differences

In [4]:
# Load the base model
# base_model = load_model("microsoft/Phi-3-mini-4k-instruct", device = device)
base_model = load_model("microsoft/phi-2", device = device)

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [5]:
# Load your fine-tuned model
# fine_tuned_model = load_model("sayanbanerjee32/multimodal-phi3-4k-instruct-llava", device = device)
fine_tuned_model = load_model("sayanbanerjee32/ms-phi2-qlora-oasst1", device = device)

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [6]:
# Compare the models
differences = compare_models(base_model, fine_tuned_model)

# Sort differences by magnitude
differences.sort(key=lambda x: x[1], reverse=True)

In [7]:
print("Top 10 layers with the largest differences:")
for name, diff in differences[:10]:
    print(f"{name}: {diff}")

Top 10 layers with the largest differences:
model.layers.31.mlp.fc2.weight: 17.955873489379883
model.layers.31.mlp.fc1.weight: 9.68554973602295
model.layers.30.mlp.fc2.weight: 7.307369709014893
model.layers.28.mlp.fc2.weight: 5.937710285186768
model.layers.30.mlp.fc1.weight: 5.635750770568848
model.layers.29.mlp.fc2.weight: 5.5101470947265625
model.layers.26.mlp.fc1.weight: 5.505995273590088
model.layers.25.mlp.fc1.weight: 4.560591220855713
model.layers.25.mlp.fc2.weight: 3.8634591102600098
model.layers.27.mlp.fc1.weight: 3.543588876724243


In [8]:
# Calculate and print statistics
diff_values = [d for _, d in differences]
print(f"\nMean difference: {np.mean(diff_values)}")
print(f"Median difference: {np.median(diff_values)}")
print(f"Max difference: {np.max(diff_values)}")
print(f"Min difference: {np.min(diff_values)}")


Mean difference: 0.44330255111584865
Median difference: 0.0
Max difference: 17.955873489379883
Min difference: 0.0


In [9]:
# Count layers with significant changes (e.g., difference > 0.01)
significant_changes = sum(1 for _, d in differences if d > 0.01)
print(f"\nLayers with significant changes (>0.01): {significant_changes} out of {len(differences)}")


Layers with significant changes (>0.01): 64 out of 453
